In [1]:
! pip install langchain langchain-groq


In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-40467?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Retail Department Manager - Clarksburg Premium Outlets

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Department Manager - Clarksburg Premium OutletsClark

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Retail Department Manager',
  'experience': 'Three years of customer-facing retail or hospitality experience, One year of leadership experience',
  'skills': 'Proficient in Microsoft Office and retail business systems, Demonstrated ability to recruit, build and lead high-performing teams, Proven ability to utilize tools for conflict resolution and employee coaching and counseling',
  'description': 'As a Nike Department Manager, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}]

In [6]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [7]:
! pip install pysqlite3


In [8]:
! pip install --upgrade pysqlite3


In [9]:
import pysqlite3 as sqlite3

# Check the SQLite version
print("SQLite version:", sqlite3.sqlite_version)

SQLite version: 3.46.1


In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

import sys
import uuid
import pysqlite3 as sqlite3  # Use pysqlite3 as your sqlite3 module
sys.modules["sqlite3"] = sqlite3  # Override the default sqlite3 module
import chromadb  # Now import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])
        




In [11]:
job = json_res[0]  # Access the first dictionary in the list
skills = job['skills']  
print("Skills:", skills)

Skills: Proficient in Microsoft Office and retail business systems, Demonstrated ability to recruit, build and lead high-performing teams, Proven ability to utilize tools for conflict resolution and employee coaching and counseling


In [12]:
links = collection.query(query_texts=job["skills"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [13]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Juanita, a business development executive at NovaCopyAI. NovaCopyAI is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of NovaCopyAI
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase NovaCopyAI's portfolio: {link_list}
        Remember you are Juanita, BDE at NovaCopyAI. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking Efficiency in Retail Operations with NovaCopyAI

Dear Hiring Manager,

I came across the Retail Department Manager role at Nike and was impressed by the emphasis on utilizing tools for conflict resolution, employee coaching, and counseling. As a business development executive at NovaCopyAI, I believe our expertise in AI and software consulting can help elevate your retail operations to the next level.

At NovaCopyAI, we specialize in developing tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team has empowered numerous enterprises with automated tools that streamline business processes, freeing up resources for strategic growth.

In the retail sector, our solutions can help you:

* Enhance customer service through AI-powered chatbots and personalized recommendations
* Optimize inventory management and supply chain logistics
* Develop data-driven insights for informed decision-making
* Automate r